In [ ]:
# Import Libraries
import json
import csv

# Import helper funcs
from Libraries.funcs import load_dict_from_json

In [ ]:
# Set models
models = [ "CohereForAI/c4ai-command-r-plus", "mistralai/miqu-70b-5_K_M", "Qwen/Qwen1.5-72B-Chat", "openai/gpt-4-0125-preview" ]

In [ ]:
# Load PCT framework from library
from Libraries.pct import questions, response_options

# Load our evaluation data
PATH_PERSONAS = './0-Data/Personas.json'
PATH_TASKS = './0-Data/Tasks.json'
PATH_EXPECTED_STANCES = './0-Data/Expected-Stance.json'
PATH_TASKS_OPINIONATED = './0-Data/Tasks-Opinionated.json'
PATH_FORCED_STANCES = './0-Data/Forced-Stances.json'
PATH_STANCE_ORIENTATION = './0-Data/Stance-Orientation.json'

with open(PATH_PERSONAS, 'r') as f:
    personas = json.load(f)

with open(PATH_TASKS, 'r') as f:
    tasks = json.load(f)

with open(PATH_EXPECTED_STANCES, 'r') as f:
    expected_stances = json.load(f)

with open(PATH_TASKS_OPINIONATED, 'r') as f:
    tasks_opinionated = json.load(f)
    
with open(PATH_FORCED_STANCES, 'r') as f:
    forced_stances = json.load(f)

with open(PATH_STANCE_ORIENTATION, 'r') as f:
    stance_orientation = json.load(f)

In [ ]:
# Political Positions
stances_econ = [ 'left', 'right' ]
stances_social = [ 'auth', 'lib' ]

# Split the questions
questions_economics = []
for q in stance_orientation.keys():
    if stance_orientation[q]['agree'] in stances_econ:
        questions_economics.append(q)

questions_social = []
for q in stance_orientation.keys():
    if stance_orientation[q]['agree'] in stances_social:
        questions_social.append(q)

# Print length
print(f"Questions Economics:\t{len(questions_economics)}")
print(f"Questions Social:\t{len(questions_social)}")

In [ ]:
for model in models:

    # Define the output path
    OUTPUT_PATH = f'./1-Results/{model}-forced-stances.json'

    # Load the results
    results = load_dict_from_json(OUTPUT_PATH)

    # ------------------------------------------------------------------------------
    # --------------------------------- Economics ----------------------------------
    # ------------------------------------------------------------------------------

    # Init the dataframe
    df_econ = []

    # Loop through results
    for key in results.keys():

        # Unpack
        question = key[2]
        expected_stance = key[0].lower()
        evaluated_stance = results[key]["evaluation"].lower()

        # Evaluation (soft, 'strongly' removed)
        expected_stance_soft = expected_stance.split(' ')[-1]
        evaluated_stance_soft = evaluated_stance.split(' ')[-1]

        # Skip if not available
        if question not in stance_orientation: continue
        if question not in questions_economics: continue

        # Map to political wording: left/right, lib/auth
        expected_stance_political = stance_orientation[question][expected_stance_soft]

        if evaluated_stance_soft != 'agree' and evaluated_stance_soft != 'disagree':
            evaluated_stance_political = 'none'
        else:
            evaluated_stance_political = stance_orientation[question][evaluated_stance_soft]

        # Append
        datum = [ question, expected_stance_political, evaluated_stance_political ]
        df_econ.append(datum)


    # Tally up the results
    df_econ_count = {}
    for q in questions_economics:
        df_econ_count[q] = {}
        for p in stances_econ:
            df_econ_count[q][p] = 0

    # Populate
    for q in questions_economics:

        # Get all results for this question
        results_q = [ d for d in df_econ if d[0] == q ]
        
        # Loop through stances
        for p in stances_econ:

            # Get all results with this expected stance
            results_q_s = [ d for d in results_q if d[1] == p ]

            # Get all results with matching expected stance and evaluated stance
            results_q_s_e = [ d for d in results_q if d[1] == p and d[2] == p ]

            # Set
            if len(results_q_s) > 0:
                df_econ_count[q][p] = len(results_q_s_e) / len(results_q_s)
            else:
                df_econ_count[q][p] = 0
            

    # Convert to table
    table_econ = []
    headers_econ = ['Question'] + stances_econ    
    for question in df_econ_count.keys():
        
        # Init row
        row = [question] + [0] * len(stances_econ)

        # Populate
        for stance in stances_econ:

            # Get the index
            index = headers_econ.index(stance)

            # Get the value
            value = df_econ_count[question][stance]

            # Set the value
            row[index] = int(100.0 * value)

        # Append
        table_econ.append(row)

    # Add rows for the averages
    row_ave = [''] * len(headers_econ)
    row_ave[0] = 'Average'
    for i in range(1, len(headers_econ)):
        col = [ row[i] for row in table_econ ]
        row_ave[i] = int(sum(col) / len(col))
    table_econ.append(row_ave)

    # Add rows for the std
    row_std = [''] * len(headers_econ)
    row_std[0] = 'Std'
    for i in range(1, len(headers_econ)):
        col = [ row[i] for row in table_econ ]
        avg = sum(col) / len(col)
        std = ( sum([ (x - avg) ** 2 for x in col ]) / len(col) ) ** 0.5
        row_std[i] = round(std, 2)
    table_econ.append(row_std)

    # Add rows for the min
    row_min = [''] * len(headers_econ)
    row_min[0] = 'Min'
    for i in range(1, len(headers_econ)):
        col = [ row[i] for row in table_econ ]
        row_min[i] = int(min(col))
    table_econ.append(row_min)

    # Add headers
    table_econ = [ headers_econ ] + table_econ

    # Save to disk
    with open(f'./1-Results/{model}-stance-counts-econ.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(table_econ)

    # ------------------------------------------------------------------------------
    # ----------------------------------- Social -----------------------------------
    # ------------------------------------------------------------------------------

    # Init the dataframe
    df_social = []

    # Loop through results
    for key in results.keys():

        # Unpack
        question = key[2]
        expected_stance = key[0].lower()
        evaluated_stance = results[key]["evaluation"].lower()

        # Evaluation (soft, 'strongly' removed)
        expected_stance_soft = expected_stance.split(' ')[-1]
        evaluated_stance_soft = evaluated_stance.split(' ')[-1]

        # Skip if not available
        if question not in stance_orientation: continue
        if question not in questions_social: continue

        # Map to political wording: left/right, lib/auth
        expected_stance_political = stance_orientation[question][expected_stance_soft]

        if evaluated_stance_soft != 'agree' and evaluated_stance_soft != 'disagree':
            evaluated_stance_political = 'none'
        else:
            evaluated_stance_political = stance_orientation[question][evaluated_stance_soft]

        # Append
        datum = [ question, expected_stance_political, evaluated_stance_political ]
        df_social.append(datum)


    # Tally up the results
    df_social_count = {}
    for q in questions_social:
        df_social_count[q] = {}
        for p in stances_social:
            df_social_count[q][p] = 0

    # Populate
    for q in questions_social:
            
        # Get all results for this question
        results_q = [ d for d in df_social if d[0] == q ]
        
        # Loop through stances
        for p in stances_social:

            # Get all results with this expected stance
            results_q_s = [ d for d in results_q if d[1] == p ]

            # Get all results with matching expected stance and evaluated stance
            results_q_s_e = [ d for d in results_q if d[1] == p and d[2] == p ]

            # Set
            if len(results_q_s) > 0:
                df_social_count[q][p] = len(results_q_s_e) / len(results_q_s)
            else:
                df_social_count[q][p] = 0

    # Convert to table
    table_social = []
    headers_social = ['Question'] + stances_social
    for question in df_social_count.keys():
        
        # Init row
        row = [question] + [0] * len(stances_social)

        # Populate
        for stance in stances_social:

            # Get the index
            index = headers_social.index(stance)

            # Get the value
            value = df_social_count[question][stance]

            # Set the value
            row[index] = int(100.0 * value)

        # Append
        table_social.append(row)

    # Add rows for the averages
    row_ave = [''] * len(headers_social)
    row_ave[0] = 'Average'
    for i in range(1, len(headers_social)):
        col = [ row[i] for row in table_social ]
        row_ave[i] = int(sum(col) / len(col))
    table_social.append(row_ave)

    # Add rows for the std
    row_std = [''] * len(headers_social)
    row_std[0] = 'Std'
    for i in range(1, len(headers_social)):
        col = [ row[i] for row in table_social ]
        avg = sum(col) / len(col)
        std = ( sum([ (x - avg) ** 2 for x in col ]) / len(col) ) ** 0.5
        row_std[i] = round(std, 2)
    table_social.append(row_std)

    # Add rows for the min
    row_min = [''] * len(headers_social)
    row_min[0] = 'Min'
    for i in range(1, len(headers_social)):
        col = [ row[i] for row in table_social ]
        row_min[i] = int(min(col))
    table_social.append(row_min)
    
    # Add headers
    table_social = [ headers_social ] + table_social

    # Save to disk
    with open(f'./1-Results/{model}-stance-counts-social.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows(table_social)


In [ ]:
# Initialize the table
table_econ_all = {}
table_social_all = {}
for q in questions_economics:
    table_econ_all[q] = [0] * len(stances_econ) * len(models)
for q in questions_social:
    table_social_all[q] = [0] * len(stances_social) * len(models)

# Combine the tables
for i, model in enumerate(models):

    # Load the tables
    table_econ_dict = {}
    with open(f'./1-Results/{model}-stance-counts-econ.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if row[0] == 'Question': continue 
            table_econ_dict[row[0]] = row[1:]

    table_social_dict = {}
    with open(f'./1-Results/{model}-stance-counts-social.csv', 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            if row[0] == 'Question': continue            
            table_social_dict[row[0]] = row[1:]

    # Append
    for q in questions_economics:
        table_econ_all[q][i*len(stances_econ):(i+1)*len(stances_econ)] = table_econ_dict[q]
    for q in questions_social:
        table_social_all[q][i*len(stances_social):(i+1)*len(stances_social)] = table_social_dict[q]

# Convert to table
table_econ_all = [ ['Question'] + [ d + '_' + models[i] for i in range(len(models)) for d in stances_econ ] ] + [ [q] + table_econ_all[q] for q in questions_economics ]
table_social_all = [ ['Question'] + [ d + '_' + models[i] for i in range(len(models)) for d in stances_social ] ] + [ [q] + table_social_all[q] for q in questions_social ]

# Sort rows based on sum of values
sorted_table_econ_all = sorted(table_econ_all[1:], key=lambda x: sum([ int(v) for v in x[1:] ]), reverse=True)
sorted_table_social_all = sorted(table_social_all[1:], key=lambda x: sum([ int(v) for v in x[1:] ]), reverse=True)
table_econ_all = [ table_econ_all[0] ] + sorted_table_econ_all
table_social_all = [ table_social_all[0] ] + sorted_table_social_all

# Add rows for the averages
row_ave = [''] * len(table_econ_all[0])
row_ave[0] = 'Average'
for i in range(1, len(table_econ_all[0])):
    col = [ float(row[i]) for row in table_econ_all[1:] ]
    row_ave[i] = int(sum(col) / len(col))
table_econ_all.append(row_ave)

# Add rows for the std
row_std = [''] * len(table_econ_all[0])
row_std[0] = 'Std'
for i in range(1, len(table_econ_all[0])):
    col = [ float(row[i]) for row in table_econ_all[1:] ]
    avg = sum(col) / len(col)
    std = ( sum([ (x - avg) ** 2 for x in col ]) / len(col) ) ** 0.5
    row_std[i] = round(std, 2)
table_econ_all.append(row_std)

# Add rows for the min
row_min = [''] * len(table_econ_all[0])
row_min[0] = 'Min'
for i in range(1, len(table_econ_all[0])):
    col = [ float(row[i]) for row in table_econ_all[1:] ]
    row_min[i] = int(min(col))
table_econ_all.append(row_min)

# Add rows for the averages
row_ave = [''] * len(table_social_all[0])
row_ave[0] = 'Average'
for i in range(1, len(table_social_all[0])):
    col = [ float(row[i]) for row in table_social_all[1:] ]
    row_ave[i] = int(sum(col) / len(col))
table_social_all.append(row_ave)

# Add rows for the std
row_std = [''] * len(table_social_all[0])
row_std[0] = 'Std'
for i in range(1, len(table_social_all[0])):
    col = [ float(row[i]) for row in table_social_all[1:] ]
    avg = sum(col) / len(col)
    std = ( sum([ (x - avg) ** 2 for x in col ]) / len(col) ) ** 0.5
    row_std[i] = round(std, 2)
table_social_all.append(row_std)

# Add rows for the min
row_min = [''] * len(table_social_all[0])
row_min[0] = 'Min'
for i in range(1, len(table_social_all[0])):
    col = [ float(row[i]) for row in table_social_all[1:] ]
    row_min[i] = int(min(col))
table_social_all.append(row_min)

# Save to disk
with open(f'./1-Results/stance-counts-econ-all.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(table_econ_all)
    
with open(f'./1-Results/stance-counts-social-all.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerows(table_social_all)